In [1]:
import os
from glob import glob
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import lsst.sims.maf.metricBundles as mb
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers

In [4]:
import sys
sys.path.append(os.path.join(os.getenv('SIMS_MAF_DIR'), 'bin.src'))
import movingObjects as moO

In [106]:
runname = 'astro_lsst_01_1017'
ddir = os.path.join('pha_split_res', runname + "_2")
outdir = runname + 'pha'
orbitFile = 'orbits/pha_5k'

In [107]:
slicer = slicers.MoObjSlicer()
slicer.readOrbits(orbitFile, Hrange=np.arange(11, 28.5, 0.5))

In [108]:
pha_splits = os.listdir('orbits/pha_split')
pha_splits

['pha_5k_00',
 'pha_5k_01',
 'pha_5k_02',
 'pha_5k_03',
 'pha_5k_04',
 'pha_5k_05',
 'pha_5k_06',
 'pha_5k_07',
 'pha_5k_08',
 'pha_5k_09']

In [109]:
# Read split results.
# ex: astro_lsst_01_1016_Discovery_Time_pha_5k_06_3_pairs_in_12_nights_MOOB.h5
strategy = ['3 pairs in 12 nights', '3 pairs in 15 nights', '3 pairs in 20 nights', 
            '3 pairs in 25 nights', '3 pairs in 30 nights', '4 pairs in 20 nights']
strategy += ['3 pairs in 15 nights trailing loss', '3 pairs in 30 nights trailing loss']
strategy += ['3 pairs in 15 nights SNReq4', '3 pairs in 30 nights SNReq4']
strategy += ['3 pairs in 15 nights trailing loss SNReq4', '3 pairs in 30 nights trailing loss SNReq4']
strategy += ['Single detection', 'Single pair']
strategy += ['3 pairs in 15 nights SNReq0']
#strategy += ['6 detections in 60 nights yr 10']
strategy += ['6 detections in 60 nights']


bundles = {}
for strat in strategy:
    bundles[strat] = {}
    for spl in pha_splits:
        filematch = os.path.join(ddir, runname + "_Discovery_Time*" + "*" + spl + "*" + 
                                 strat.replace(' ', '_') + "*h5")
        try:
            filename = glob(filematch)[0]        
        except:
            filematch = os.path.join(ddir, runname + "*" + "*" + spl + "*" +
                                    strat.replace(' ', '_') + "*h5")
            print('new filematch', filematch)
            filename = glob(filematch)[0]
        bundles[strat][spl] = mb.createEmptyMoMetricBundle()
        bundles[strat][spl].read(filename)
        Hrange = bundles[strat][spl].slicer.Hrange
        bundles[strat][spl].slicer = slicer
        bundles[strat][spl].slicer.Hrange = Hrange

new filematch pha_split_res/astro_lsst_01_1017_2/astro_lsst_01_1017**pha_5k_00*3_pairs_in_12_nights*h5


IndexError: list index out of range

In [101]:
# Join results.
times = np.arange(59579, 65055.15+5, 10)
summaryMetrics = [metrics.MoCompletenessAtTimeMetric(times=times, Hval=22,cumulative=False, Hindex=0.33),
                  metrics.MoCompletenessAtTimeMetric(times=times, Hval=22, cumulative=True, Hindex=0.33)]
summaryMetrics2 = [metrics.MoCompletenessMetric(cumulative=False, Hindex=0.33),
                  metrics.MoCompletenessMetric(cumulative=True, Hindex=0.33)]

timebundles = {}
for strat in strategy:
    filematch = os.path.join(ddir, runname + "_Discovery_Time*" + "*pha_5k_00*" + strat.replace(' ', '_') + "*h5")
    flagT = True
    try:
        filename = glob(filematch)[0]
    except:
        filematch = os.path.join(ddir, runname + "*" + "*" + '*pha_5k_00*' +
                                    strat.replace(' ', '_') + "*h5")
        flagT = False
        filename = glob(filematch)[0]
    timebundles[strat] = mb.createEmptyMoMetricBundle()
    timebundles[strat].read(filename)
    timebundles[strat].fileRoot = timebundles[strat].fileRoot.replace('_00', '')
    Hrange1 = timebundles[strat].slicer.Hrange
    timebundles[strat].slicer = slicer
    timebundles[strat].slicer.Hrange = Hrange1
    mvals = np.zeros((len(slicer.orbits), len(Hrange1)), float)
    mvalsMask = np.ones((len(slicer.orbits), len(Hrange1)), dtype=bool)
    for spl in pha_splits:
        if np.any(np.not_equal(bundles[strat][spl].slicer.Hrange, Hrange1)):
            raise ValueError('Incompatible results, Hrange varies')
        mvals = mvals + bundles[strat][spl].metricValues.filled(0)
        mvalsMask = np.where(mvalsMask & bundles[strat][spl].metricValues.mask, True, False)
    timebundles[strat].metricValues = ma.MaskedArray(data=mvals, mask=mvalsMask, fill_value=0)
    timebundles[strat].metadata = 'PHA ' + strat
    timebundles[strat].runname = runname
    if flagT:
        timebundles[strat].setSummaryMetrics(summaryMetrics)
        timebundles[strat].computeSummaryStats()
    else:
        timebundles[strat].setSummaryMetrics(summaryMetrics2)
        timebundles[strat].computeSummaryStats()
    timebundles[strat].write()

In [94]:
# old runs : 59580 start 
# v4 runs: 59853 start
#59853 + 365 *10
59580 + 365 * 12
59853 -59580

273

In [95]:
#10 years, old runs
diff = np.abs(times-(63230.50 + 0))
yr10idx = np.where(diff == np.min(diff))[0]
#12 years, old runs
diff = np.abs(times-(63960 + 0))
yr12idx = np.where(diff == np.min(diff))[0]
print(yr10idx, times[yr10idx], yr12idx, times[yr12idx])

[365] [ 63229.] [438] [ 63959.]


In [45]:
# astro-lsst-01-1016
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][yr12idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][yr12idx])

3 pairs in 12 nights ['H <=22.0 @ 63959.00'] [ 0.74128294]
3 pairs in 15 nights ['H <=22.0 @ 63959.00'] [ 0.75043703]
3 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.7584721]
3 pairs in 25 nights ['H <=22.0 @ 63959.00'] [ 0.76503545]
3 pairs in 30 nights ['H <=22.0 @ 63959.00'] [ 0.77098024]
4 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.7019879]
3 pairs in 15 nights trailing loss ['H <=22.0 @ 63959.00'] [ 0.74929041]
3 pairs in 30 nights trailing loss ['H <=22.0 @ 63959.00'] [ 0.77011237]
3 pairs in 15 nights SNReq4 ['H <=22.0 @ 63959.00'] [ 0.77653691]
3 pairs in 30 nights SNReq4 ['H <=22.0 @ 63959.00'] [ 0.79450148]
3 pairs in 15 nights trailing loss SNReq4 ['H <=22.0 @ 63959.00'] [ 0.77653691]
3 pairs in 30 nights trailing loss SNReq4 ['H <=22.0 @ 63959.00'] [ 0.79450148]
Single detection ['H <=22.0 @ 63959.00'] [ 0.88771766]
Single pair ['H <=22.0 @ 63959.00'] [ 0.85574515]


In [44]:
# astro-lsst-01_1016 @ time 10 years
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][yr10idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][yr10idx])

3 pairs in 12 nights ['H <=22.0 @ 63229.00'] [ 0.70072872]
3 pairs in 15 nights ['H <=22.0 @ 63229.00'] [ 0.71008376]
3 pairs in 20 nights ['H <=22.0 @ 63229.00'] [ 0.71860778]
3 pairs in 25 nights ['H <=22.0 @ 63229.00'] [ 0.72563463]
3 pairs in 30 nights ['H <=22.0 @ 63229.00'] [ 0.7321288]
4 pairs in 20 nights ['H <=22.0 @ 63229.00'] [ 0.65956568]
3 pairs in 15 nights trailing loss ['H <=22.0 @ 63229.00'] [ 0.7087657]
3 pairs in 30 nights trailing loss ['H <=22.0 @ 63229.00'] [ 0.73161255]
3 pairs in 15 nights SNReq4 ['H <=22.0 @ 63229.00'] [ 0.7388797]
3 pairs in 30 nights SNReq4 ['H <=22.0 @ 63229.00'] [ 0.75842672]
3 pairs in 15 nights trailing loss SNReq4 ['H <=22.0 @ 63229.00'] [ 0.7388797]
3 pairs in 30 nights trailing loss SNReq4 ['H <=22.0 @ 63229.00'] [ 0.75842672]
Single detection ['H <=22.0 @ 63229.00'] [ 0.86111512]
Single pair ['H <=22.0 @ 63229.00'] [ 0.82504625]


In [23]:
# astro-lsst-01_1016 @ time 15 years
diff = np.abs(times-65055.5)
idx = np.where(diff == np.min(diff))[0]
idx, times[idx]
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][idx])

3 pairs in 12 nights ['H <=22.0 @ 65059.00'] [ 0.78152201]
3 pairs in 15 nights ['H <=22.0 @ 65059.00'] [ 0.78909266]
3 pairs in 20 nights ['H <=22.0 @ 65059.00'] [ 0.79669691]
3 pairs in 25 nights ['H <=22.0 @ 65059.00'] [ 0.80288782]
3 pairs in 30 nights ['H <=22.0 @ 65059.00'] [ 0.80827317]
4 pairs in 20 nights ['H <=22.0 @ 65059.00'] [ 0.74443137]


In [52]:
# astro-lsst-01-1015 yr 10
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][yr10idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][yr10idx])

3 pairs in 12 nights ['H <=22.0 @ 63229.00'] [ 0.69957278]
3 pairs in 15 nights ['H <=22.0 @ 63229.00'] [ 0.71285879]
3 pairs in 20 nights ['H <=22.0 @ 63229.00'] [ 0.72676014]
3 pairs in 25 nights ['H <=22.0 @ 63229.00'] [ 0.73830412]
3 pairs in 30 nights ['H <=22.0 @ 63229.00'] [ 0.74658048]
4 pairs in 20 nights ['H <=22.0 @ 63229.00'] [ 0.64279197]
3 pairs in 15 nights trailing loss ['H <=22.0 @ 63229.00'] [ 0.70973404]
3 pairs in 30 nights trailing loss ['H <=22.0 @ 63229.00'] [ 0.7456052]
3 pairs in 15 nights SNReq4 ['H <=22.0 @ 63229.00'] [ 0.74741276]
3 pairs in 30 nights SNReq4 ['H <=22.0 @ 63229.00'] [ 0.77802755]
3 pairs in 15 nights trailing loss SNReq4 ['H <=22.0 @ 63229.00'] [ 0.74741276]
3 pairs in 30 nights trailing loss SNReq4 ['H <=22.0 @ 63229.00'] [ 0.77802755]
Single detection ['H <=22.0 @ 63229.00'] [ 0.88823557]
Single pair ['H <=22.0 @ 63229.00'] [ 0.85722786]


In [53]:
# astro-lsst-01-1015 yr 12
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][yr12idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][yr12idx])

3 pairs in 12 nights ['H <=22.0 @ 63959.00'] [ 0.74067693]
3 pairs in 15 nights ['H <=22.0 @ 63959.00'] [ 0.75258709]
3 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.76580289]
3 pairs in 25 nights ['H <=22.0 @ 63959.00'] [ 0.77648192]
3 pairs in 30 nights ['H <=22.0 @ 63959.00'] [ 0.783748]
4 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.68678541]
3 pairs in 15 nights trailing loss ['H <=22.0 @ 63959.00'] [ 0.74969784]
3 pairs in 30 nights trailing loss ['H <=22.0 @ 63959.00'] [ 0.78202353]
3 pairs in 15 nights SNReq4 ['H <=22.0 @ 63959.00'] [ 0.78593362]
3 pairs in 30 nights SNReq4 ['H <=22.0 @ 63959.00'] [ 0.81287801]
3 pairs in 15 nights trailing loss SNReq4 ['H <=22.0 @ 63959.00'] [ 0.78593362]
3 pairs in 30 nights trailing loss SNReq4 ['H <=22.0 @ 63959.00'] [ 0.81287801]
Single detection ['H <=22.0 @ 63959.00'] [ 0.91061248]
Single pair ['H <=22.0 @ 63959.00'] [ 0.88375897]


In [60]:
# astro-lsst-01-1017 yr 10
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][yr10idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][yr10idx])

3 pairs in 12 nights ['H <=22.0 @ 63229.00'] [ 0.71641057]
3 pairs in 15 nights ['H <=22.0 @ 63229.00'] [ 0.7268258]
3 pairs in 20 nights ['H <=22.0 @ 63229.00'] [ 0.73816745]
3 pairs in 25 nights ['H <=22.0 @ 63229.00'] [ 0.74797051]
3 pairs in 30 nights ['H <=22.0 @ 63229.00'] [ 0.75565303]
4 pairs in 20 nights ['H <=22.0 @ 63229.00'] [ 0.67112075]
3 pairs in 15 nights trailing loss ['H <=22.0 @ 63229.00'] [ 0.72779583]
3 pairs in 30 nights trailing loss ['H <=22.0 @ 63229.00'] [ 0.75592634]
3 pairs in 15 nights SNReq4 ['H <=22.0 @ 63229.00'] [ 0.75758425]
3 pairs in 30 nights SNReq4 ['H <=22.0 @ 63229.00'] [ 0.78346429]
3 pairs in 15 nights trailing loss SNReq4 ['H <=22.0 @ 63229.00'] [ 0.75758425]
3 pairs in 30 nights trailing loss SNReq4 ['H <=22.0 @ 63229.00'] [ 0.78346429]
Single detection ['H <=22.0 @ 63229.00'] [ 0.88395827]
Single pair ['H <=22.0 @ 63229.00'] [ 0.85526591]


In [61]:
# astro-lsst-01-1017 yr 12
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][yr12idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][yr12idx])

3 pairs in 12 nights ['H <=22.0 @ 63959.00'] [ 0.75494339]
3 pairs in 15 nights ['H <=22.0 @ 63959.00'] [ 0.76501288]
3 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.77603137]
3 pairs in 25 nights ['H <=22.0 @ 63959.00'] [ 0.7836424]
3 pairs in 30 nights ['H <=22.0 @ 63959.00'] [ 0.78993808]
4 pairs in 20 nights ['H <=22.0 @ 63959.00'] [ 0.71164474]
3 pairs in 15 nights trailing loss ['H <=22.0 @ 63959.00'] [ 0.76666834]
3 pairs in 30 nights trailing loss ['H <=22.0 @ 63959.00'] [ 0.79191119]
3 pairs in 15 nights SNReq4 ['H <=22.0 @ 63959.00'] [ 0.7937622]
3 pairs in 30 nights SNReq4 ['H <=22.0 @ 63959.00'] [ 0.81551207]
3 pairs in 15 nights trailing loss SNReq4 ['H <=22.0 @ 63959.00'] [ 0.7937622]
3 pairs in 30 nights trailing loss SNReq4 ['H <=22.0 @ 63959.00'] [ 0.81551207]
Single detection ['H <=22.0 @ 63959.00'] [ 0.90729345]
Single pair ['H <=22.0 @ 63959.00'] [ 0.88154957]


In [102]:
# minion_1016
Hidx = np.where(Hrange ==22)
for strat in strategy:
    try:
        print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][yr10idx],
             timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][yr10idx])
    except:
        print(strat, timebundles[strat].summaryValues['CumulativeCompleteness']['name'][Hidx],
             timebundles[strat].summaryValues['CumulativeCompleteness']['value'][Hidx])

3 pairs in 12 nights ['H <=22.0 @ 63229.00'] [ 0.69440213]
3 pairs in 15 nights ['H <=22.0 @ 63229.00'] [ 0.70065299]
3 pairs in 20 nights ['H <=22.0 @ 63229.00'] [ 0.70842489]
3 pairs in 25 nights ['H <=22.0 @ 63229.00'] [ 0.71515535]
3 pairs in 30 nights ['H <=22.0 @ 63229.00'] [ 0.72162124]
4 pairs in 20 nights ['H <=22.0 @ 63229.00'] [ 0.65726505]
3 pairs in 15 nights trailing loss ['H <=22.0 @ 63229.00'] [ 0.70047938]
3 pairs in 30 nights trailing loss ['H <=22.0 @ 63229.00'] [ 0.72112483]
3 pairs in 15 nights SNReq4 ['H <=22.0 @ 63229.00'] [ 0.73150312]
3 pairs in 30 nights SNReq4 ['H <=22.0 @ 63229.00'] [ 0.75141154]
3 pairs in 15 nights trailing loss SNReq4 ['H <=22.0 @ 63229.00'] [ 0.73150312]
3 pairs in 30 nights trailing loss SNReq4 ['H <=22.0 @ 63229.00'] [ 0.75141154]
Single detection ['H <=22.0 @ 63229.00'] [ 0.84789897]
Single pair ['H <=22.0 @ 63229.00'] [ 0.81416481]
3 pairs in 15 nights SNReq0 ['H <=22.0 @ 63229.00'] [ 0.9356]
6 detections in 60 nights yr 10 ['H <= 22

In [70]:
# astro-lsst-01_2013
for strat in strategy:
    print(strat, timebundles[strat].summaryValues['CumulativeCompleteness@Time']['name'][yr10idx],
         timebundles[strat].summaryValues['CumulativeCompleteness@Time']['value'][yr10idx])

3 pairs in 12 nights ['H <=22.0 @ 63499.00'] [ 0.66976778]
3 pairs in 15 nights ['H <=22.0 @ 63499.00'] [ 0.6773945]
3 pairs in 20 nights ['H <=22.0 @ 63499.00'] [ 0.68375953]
3 pairs in 25 nights ['H <=22.0 @ 63499.00'] [ 0.68904486]
3 pairs in 30 nights ['H <=22.0 @ 63499.00'] [ 0.69425086]
4 pairs in 20 nights ['H <=22.0 @ 63499.00'] [ 0.635764]
3 pairs in 15 nights trailing loss ['H <=22.0 @ 63499.00'] [ 0.67605396]
3 pairs in 30 nights trailing loss ['H <=22.0 @ 63499.00'] [ 0.69507109]
3 pairs in 15 nights SNReq4 ['H <=22.0 @ 63499.00'] [ 0.70347042]
3 pairs in 30 nights SNReq4 ['H <=22.0 @ 63499.00'] [ 0.72029171]
Single detection ['H <=22.0 @ 63499.00'] [ 0.82004253]
Single pair ['H <=22.0 @ 63499.00'] [ 0.78185912]
